In [ ]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:8 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu 

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-05-20 01:36:27--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2021-05-20 01:36:28 (6.64 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Tools_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   15785389|R2UM5QMHBHC90Q|B00H5U9ZD6|     115362950|WallPeg 12 sq ft ...|           Tools|          5|            0|          0|   N|                Y|Great organizer, ...|Very nice. Will o...| 2015-08-31|
|         US|   47910848|  RF0D1LEIF6L7|B001TJGCS0|     570955425|Nite Ize Nite Daw...|           Tools|          4|    

In [ ]:
# 1. Reviews with greater or equal to 20 reviews
over_20_df = df.filter(df["total_votes"] >= 20)
over_20_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   47749608| RXAHWIC1584UQ|B00NKSMPZW|     824618679|ClearArmor 141001...|           Tools|          5|           20|         23|   N|                Y|          Five Stars|geart product :) ...| 2015-08-31|
|         US|   53082915| RSBELAIC899DO|B00LPFIPPQ|     848632692|WEN 6510 Oscillating|           Tools|          5|    

In [ ]:
# 2. Now helpful / total votes greater than 50%
above_50_df = over_20_df.filter(over_20_df["helpful_votes"]/over_20_df["total_votes"] >= 0.5)
above_50_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   47749608| RXAHWIC1584UQ|B00NKSMPZW|     824618679|ClearArmor 141001...|           Tools|          5|           20|         23|   N|                Y|          Five Stars|geart product :) ...| 2015-08-31|
|         US|   53082915| RSBELAIC899DO|B00LPFIPPQ|     848632692|WEN 6510 Oscillating|           Tools|          5|    

In [ ]:
# 3. Paid vines
paid_df = above_50_df.filter(above_50_df["vine"] == "Y")
paid_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   21084288|R3QIQRG107AP19|B00OJ72LHK|     369815253|BLACK+DECKER BDEJ...|           Tools|          5|           38|         41|   Y|                N|Essential home ow...|We are not pros. ...| 2015-08-27|
|         US|   52512530|R1ZNBHGCYE8VN1|B00OJ72LHK|     369815253|BLACK+DECKER BDEJ...|           Tools|          5|    

In [ ]:
# 4. Unpaid vines
unpaid_df = above_50_df.filter(above_50_df["vine"] == "N")
unpaid_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   47749608| RXAHWIC1584UQ|B00NKSMPZW|     824618679|ClearArmor 141001...|           Tools|          5|           20|         23|   N|                Y|          Five Stars|geart product :) ...| 2015-08-31|
|         US|   53082915| RSBELAIC899DO|B00LPFIPPQ|     848632692|WEN 6510 Oscillating|           Tools|          5|    

In [ ]:
# 5. Now the Metrics

# Paid Reviews First
total_paid_reviews = paid_df.count()
five_star_paid_reviews = paid_df.filter(paid_df["star_rating"] == "5").count()
paid_five_star_percentage = (five_star_paid_reviews / total_paid_reviews) *100
print('Vine reviews: {}, Five star Vine reviews: {}, Percentage of 5-star Vine reviews: {}'.format(total_paid_reviews ,five_star_paid_reviews, paid_five_star_percentage))

Vine reviews: 285, Five star Vine reviews: 163, Percentage of 5-star Vine reviews: 57.19298245614035


In [ ]:
# Unpaid Reviews
total_unpaid_reviews = unpaid_df.count()
five_star_unpaid_reviews = unpaid_df.filter(unpaid_df["star_rating"] == "5").count()
unpaid_five_star_percentage = (five_star_unpaid_reviews / total_unpaid_reviews) *100
print('Non-Vine reviews: {}, Five star non-Vine reviews: {}, Percentage of 5-star non-Vine reviews: {}'.format(total_unpaid_reviews ,five_star_unpaid_reviews, unpaid_five_star_percentage))

Non-Vine reviews: 31545, Five star non-Vine reviews: 14614, Percentage of 5-star non-Vine reviews: 46.32746869551435
